## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001CCDF58F6D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001CCDF5C8B80>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, My name is Aditya and I am learning GENAI")])

AIMessage(content="Hi Aditya, it's nice to meet you!\n\nLearning about Generative AI (GENAI) is exciting! It's a rapidly developing field with a lot of potential. \n\nWhat specifically are you interested in learning about GENAI? \n\nFor example, are you curious about:\n\n* **Different types of GENAI models** (like text-based models like me, image generators, music composers, etc.)?\n* **How GENAI models are trained?**\n* **The ethical considerations surrounding GENAI?**\n* **Potential applications of GENAI in various industries?**\n\nKnowing your interests will help me provide you with more relevant and helpful information.\n\nGood luck with your learning journey!  I'm here to help in any way I can.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 163, 'prompt_tokens': 21, 'total_tokens': 184, 'completion_time': 0.296363636, 'prompt_time': 0.002254586, 'queue_time': 0.242257844, 'total_time': 0.298618222}, 'model_name': 'Gemma2-9b-It', 'system_fingerpri

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi, My name is Aditya and I am learning GENAI"),
        AIMessage(content="Hello Aditya! That's great to hear. What specific aspects of GENAI are you interested in learning about?"),
        HumanMessage(content="Hey What's is my name and what is currently I am doing?")
    ]
)

AIMessage(content="Your name is Aditya, and right now you are having a conversation with me, an AI assistant.  You're learning about GENAI! 😊 \n\nIs there anything specific you'd like to know about GENAI? I'm happy to help in any way I can.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 69, 'total_tokens': 130, 'completion_time': 0.110909091, 'prompt_time': 0.003672751, 'queue_time': 0.25101206200000004, 'total_time': 0.114581842}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--2771d1ae-62c5-4325-a284-7e29ba23bf69-0', usage_metadata={'input_tokens': 69, 'output_tokens': 61, 'total_tokens': 130})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [6]:
config = {"configurable": {"session_id": "chat1"}}

In [7]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is Aditya and I am learning GENAI"),
    ],
    config=config
)

In [8]:
response.content

"Hi Aditya, it's great to meet you!  Learning about Generative AI (GENAI) is exciting. It's a rapidly evolving field with tons of potential. \n\nWhat specifically are you interested in learning about GENAI?  \n\nPerhaps you'd like to know more about:\n\n* **Different types of GENAI models:** Like text generators (like me!), image creators, music composers, etc.\n* **How GENAI works:** The underlying technology and concepts like deep learning and transformers.\n* **Applications of GENAI:**  How it's being used in areas like art, writing, research, and more.\n* **Ethical considerations:** The potential biases and risks associated with GENAI.\n\nTell me more about your goals, and I'll do my best to help you on your GENAI learning journey!\n"

In [9]:
with_message_history.invoke(
    [
        HumanMessage(content="Hey What's is my name and what is currently I am doing?")
    ],
    config=config
)

AIMessage(content='You told me your name is Aditya.  \n\nYou are currently having a conversation with me, an AI assistant, and learning about Generative AI! 😊  \n\n\n\nLet me know if you have any other questions.\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 220, 'total_tokens': 266, 'completion_time': 0.083636364, 'prompt_time': 0.00869615, 'queue_time': 0.246805243, 'total_time': 0.092332514}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--cbd5a4d2-4527-4517-b6f2-054239782d7b-0', usage_metadata={'input_tokens': 220, 'output_tokens': 46, 'total_tokens': 266})

In [10]:
### Change the config ---> session_id
config1 = {"configurable": {"session_id": "chat2"}}
response = with_message_history.invoke(
    [
        HumanMessage(content="Hey What's is my name and what is currently I am doing?"),
    ],
    config=config1
)

In [11]:
response.content

"As an AI, I have no memory of past conversations and no way of knowing your name or what you're doing.\n\nIf you'd like to tell me your name, I'd be happy to know! 😊  And I'm always ready for a chat about whatever's on your mind.\n"

In [12]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is Adam and I am a software engineer in GENAI"),
    ],
    config=config1
)
response.content

"Hi Adam! It's great to meet you.  \n\nBeing a software engineer in GENAI must be fascinating. What kind of projects are you working on? Do you have any cool projects you'd like to tell me about?  I'm always eager to learn more about the world of AI.\n"

In [14]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Hey What's is my name and who am I?"),
    ],
    config=config1
)
response.content

"You told me your name is Adam, and that you're a software engineer in GENAI.  \n\nIs there anything else you'd like to tell me about yourself?  I'm always happy to chat! 😊  \n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [19]:
chain.invoke({"messages": [HumanMessage(content="Hi, My name is Aditya and I am learning GENAI")]})

AIMessage(content="Hi Aditya, it's great to meet you!  \n\nI'm glad you're learning about Generative AI (GENAI). It's a fascinating field with a lot of potential.  \n\nWhat can I help you with today? Do you have any specific questions about GENAI, or are you looking for general information?\n\nI can tell you about:\n\n* **What GENAI is and how it works**\n* **Different types of GENAI models (like text generators, image creators, etc.)**\n* **Examples of GENAI in action**\n* **The ethical considerations surrounding GENAI**\n* **Resources for learning more about GENAI**\n\n\nJust let me know what you're interested in! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 157, 'prompt_tokens': 37, 'total_tokens': 194, 'completion_time': 0.285454545, 'prompt_time': 0.002415775, 'queue_time': 0.243760485, 'total_time': 0.28787032}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--93325

In [20]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [21]:
config2 = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is Aditya and I am learning GENAI"),
    ],
    config=config2
)
response.content

"Hello Aditya! It's great to meet you.  \n\nI'm happy to help you learn about Generative AI (GENAI). What can I tell you about it? \n\nDo you have any specific questions in mind? \n\nFor example, are you curious about:\n\n* **What GENAI is and how it works?**\n* **Different types of GENAI models (like text generators, image generators, etc.)?**\n* **Examples of GENAI applications?**\n* **The ethical considerations surrounding GENAI?**\n\n\nLet me know where you'd like to start, and I'll do my best to provide helpful information! 😊  \n\n"